In [1]:
import plotly.express as plt 
import pandas as pd
import os
import numpy as np
import sys 
sys.path.append('./modules/')
from mva import *

In [2]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [3]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

In [4]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [5]:
print(meanWaits)

     DELAY     SWAP_IN       CPU        IO1          IO2   N
0      0.0    0.000000  0.000000   0.000000     0.000000   0
1   5000.0  210.000000  2.700000  40.000000   180.000000   1
2   5000.0  223.824451  2.928527  43.260188   205.391850   2
3   5000.0  238.389124  3.177571  46.793396   235.823993   3
4   5000.0  253.533652  3.446068  50.579796   272.282610   4
5   5000.0  269.032970  3.731638  54.581095   315.853912   5
6   5000.0  284.600954  4.030402  58.738515   367.680155   6
7   5000.0  299.904587  4.336961  62.973254   428.890837   7
8   5000.0  314.589239  4.644616  67.190305   500.511697   8
9   5000.0  328.312511  4.945852  71.285815   583.361328   9
10  5000.0  340.780878  5.233042  75.157177   677.952217  10
11  5000.0  351.781406  5.499257  78.714047   784.416577  11
12  5000.0  361.201359  5.739004  81.887950   902.474954  12
13  5000.0  369.031668  5.948742  84.638366  1031.456793  13
14  5000.0  375.354754  6.127068  86.954192  1170.369616  14
15  5000.0  380.321190  

In [6]:


activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
print(activeTimes)


     N          CPU     SWAP_IN          IO1           IO2
0    0     0.000000    0.000000     0.000000      0.000000
1    1   675.000000  525.000000   650.000000   1125.000000
2    2   732.131662  559.561129   702.978057   1283.699060
3    3   794.392798  595.972811   760.392684   1473.899954
4    4   861.517004  633.834130   821.921680   1701.766314
5    5   932.909505  672.582426   886.942799   1974.086948
6    6  1007.600472  711.502385   954.500871   2298.000967
7    7  1084.240242  749.761468  1023.315381   2680.567734
8    8  1161.154028  786.473098  1091.842463   3128.198104
9    9  1236.462960  820.781277  1158.394499   3646.008298
10  10  1308.260497  851.952195  1221.304120   4237.201358
11  11  1374.814216  879.453516  1279.103264   4902.603606
12  12  1434.751032  903.003398  1330.679193   5640.468464
13  13  1487.185540  922.579169  1375.373453   6446.604955
14  14  1531.766887  938.386884  1413.005626   7314.810103
15  15  1568.642694  950.802975  1443.824881   8237.5124

In [7]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    #data = fig.to_image("png")
    #f = open("./Images/{}.png".format(name),"+wb")
    #f.write(data)
    #f.close()
    fig.show()
pass

In [8]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [9]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [10]:
produce_graph_and_save(meanClients,"meanClients","n")

In [11]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [12]:
produce_graph_and_save(activeTimes,"active_times","ms",True)

In [13]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [14]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

# Markov chain model

In [15]:
mva = MVA(matrix,[5000,0,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)
mva()
utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]


In [16]:
produce_graph_and_save(utilization,"Utilizations_markov","B/T")

In [17]:
produce_graph_and_save(meanWaits,"meanWaits_markov","ms",True)

In [18]:
produce_graph_and_save(meanClients,"meanClients_markov","n")

In [19]:
produce_graph_and_save(throughputs,"throughputs_markov","C/T")

In [20]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [21]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")